In [ ]:
!pip install transformers==4.29.2
!pip install sentencepiece
!pip install torch==2.2.0

In [ ]:
!pip install cpm_kernels ujson tensorboard datasets bitsandbytes scipy wandb peft==0.3.0 uvicorn==0.21.1 fastapi==0.95.1 pydantic==1.10.7

In [ ]:
from transformers import AutoTokenizer, AutoModel
import json

def get_dialogue_history(dialogue_history_list: list):

    dialogue_history_tmp = []
    for item in dialogue_history_list:
        if item['role'] == 'counselor':
            text = '咨询师：'+ item['content']
        else:
            text = '来访者：'+ item['content']
        dialogue_history_tmp.append(text)

    dialogue_history = '\n'.join(dialogue_history_tmp)

    return dialogue_history + '\n' + '咨询师：'

def get_instruction(dialogue_history):
    instruction = f'''现在你扮演一位专业的心理咨询师，你具备丰富的心理学和心理健康知识。你擅长运用多种心理咨询技巧，例如认知行为疗法原则、动机访谈技巧和解决问题导向的短期疗法。以温暖亲切的语气，展现出共情和对来访者感受的深刻理解。以自然的方式与来访者进行对话，避免过长或过短的回应，确保回应流畅且类似人类的对话。提供深层次的指导和洞察，使用具体的心理概念和例子帮助来访者更深入地探索思想和感受。避免教导式的回应，更注重共情和尊重来访者的感受。根据来访者的反馈调整回应，确保回应贴合来访者的情境和需求。请为以下的对话生成一个回复。

对话：
{dialogue_history}'''

    return instruction


tokenizer = AutoTokenizer.from_pretrained('qiuhuachuan/PsyChat', trust_remote_code=True) # MeChat
model = AutoModel.from_pretrained('qiuhuachuan/PsyChat', trust_remote_code=True).half().cuda()
model = model.eval()

with open('./auto_eval_chat_data.json') as f:
    data = json.load(f)

    for i in range(100):
        dialogue_history_with_label = []
        dialogue_history_with_output = []

        dialogue = data[i]['messages']
        for j in range(0,10,2):
            usr_msg = dialogue[j]['content']
            dialogue_history_with_label.append({
                'role': 'client',
                'content': usr_msg
            })
            dialogue_history_with_output.append({
                'role': 'client',
                'content': usr_msg
            })

            dialogue_history = get_dialogue_history(dialogue_history_list=dialogue_history_with_output)
            instruction = get_instruction(dialogue_history=dialogue_history)
            response, history = model.chat(tokenizer, instruction, history=[], temperature=0.8, top_p=0.8)
            dialogue_history_with_output.append({
                'role': 'counselor',
                'content': response
            })
            print(dialogue_history_with_output)
            dialogue[j+1]['PsyChat_response_history_with_output'] = response
            print(data)

            dialogue_history = get_dialogue_history(dialogue_history_list=dialogue_history_with_label)
            instruction = get_instruction(dialogue_history=dialogue_history)
            response, history = model.chat(tokenizer, instruction, history=[], temperature=0.8, top_p=0.8)
            dialogue_history_with_label.append({
                'role': 'counselor',
                'content': dialogue[j+1]['content']
            })
            dialogue[j+1]['PsyChat_response_history_with_label'] = response


with open('./PsyChat_result.json', 'w') as res_f:
    json.dump(data, res_f, ensure_ascii=False, indent=2)

